In [1]:
import numpy as np 
import composite as cp 

a = cp.layer()
a.set_by_E(140,8.6,0.35,5.0)
a.S
a.Q

a.get_status()

name:	HT3/5224
t:	 0.000125
angle:	 0
Q:	 [[1.41061488e+11 3.03282199e+09 0.00000000e+00]
 [3.03282199e+09 8.66520567e+09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e+09]]
S:	 [[ 7.14285714e-12 -2.50000000e-12  0.00000000e+00]
 [-2.50000000e-12  1.16279070e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e-10]]
Q_:	 [[1.41061488e+11 3.03282199e+09 0.00000000e+00]
 [3.03282199e+09 8.66520567e+09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e+09]]


In [2]:
b = cp.layer()
b.set_angle_degree(0)
b.get_status()

name:	HT3/5224
t:	 0.000125
angle:	 0.0
Q:	 [[1.41061488e+11 3.03282199e+09 0.00000000e+00]
 [3.03282199e+09 8.66520567e+09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e+09]]
S:	 [[ 7.14285714e-12 -2.50000000e-12  0.00000000e+00]
 [-2.50000000e-12  1.16279070e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e-10]]
Q_:	 [[1.41061488e+11 3.03282199e+09 0.00000000e+00]
 [3.03282199e+09 8.66520567e+09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e+09]]
